In [24]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from torch.utils.data import random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [25]:
training_pvs = torch.load("train_pvs.pt")
val_pvs = torch.load("val_pvs.pt")
training_sgs = torch.load("train_sgs.pt")
val_sgs = torch.load("val_sgs.pt")
train_composition = torch.load("train_composition.pt")
val_composition = torch.load("val_composition.pt")

train_composition2D = torch.load("train_composition2D.pt")
val_composition2D = torch.load("val_composition2D.pt")

In [26]:
training_pvs = training_pvs.to(device)
training_sgs = training_sgs.to(device)
val_pvs = val_pvs.to(device)
val_sgs = val_sgs.to(device)

train_composition = train_composition.to(device).float()
val_composition = val_composition.to(device).float()

train_composition2D = train_composition2D.to(device).float()
val_composition2D = val_composition2D.to(device).float()

In [27]:
mean = train_composition.mean()
std  = train_composition.std()

train_composition = (train_composition - mean ) / std
val_composition = (val_composition - mean) / std

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

class XRD_and_composition(Dataset):
    def __init__(self, xrd, composition, targets):
        assert xrd.size(0) == composition.size(0) == targets.size(0), "The number of elements in both tensor sets should be the same"
        self.xrd = xrd
        self.composition = composition
        self.targets = targets

    def __len__(self):
        return self.xrd.size(0)

    def __getitem__(self, index):
        return (self.xrd[index], self.composition[index], self.targets[index])

In [35]:
len(val_composition.shape)

3

In [78]:
#shrink the training data for sandboxing 
fraction_of_total_data = 1
amt_of_data = int(fraction_of_total_data * train_composition.shape[0])

train_dataset = XRD_and_composition(training_pvs[:amt_of_data], train_composition[:amt_of_data], training_sgs[:amt_of_data])
val_dataset = TensorDataset(val_pvs, val_composition, val_sgs)

In [79]:
# Create DataLoaders for train and validation sets
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)  

In [80]:
class XRD_convnet(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(XRD_convnet, self).__init__()
        self.flatten = nn.Flatten()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels, 80, kernel_size = 100, stride=5),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(80),
            nn.Conv1d(80, 80, 50, stride=5),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(80),
            nn.Conv1d(80, 80, 25, stride=2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(80),
        )

         # Calculate flattened_size dynamically
        self.flattened_size = self._get_flattened_size(input_shape=(1, in_channels, 8500))

        self.MLP = nn.Sequential(
            nn.Linear(self.flattened_size, 2300),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2300, 1150),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1150, output_dim)
        )

       

    def _get_flattened_size(self, input_shape):
        dummy_input = torch.zeros(input_shape)
        with torch.no_grad():
            dummy_output = self.conv_layers(dummy_input)
        return int(np.prod(dummy_output.shape))
    
    def forward(self, x,): 

        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.MLP(x)

        return x


class composition_MLP(nn.Module):
    def __init__(self):
        super(composition_MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.composition_net = nn.Sequential(
            nn.Linear(100, 230),
            nn.ReLU(),
            nn.BatchNorm1d(230),  
            nn.Linear(230, 230),
            nn.ReLU(),
            nn.BatchNorm1d(230),  
            nn.Linear(230, 230),
            nn.ReLU(),
            nn.BatchNorm1d(230),  
            nn.Linear(230, 230),
            nn.ReLU(),
        )

    def forward(self, c): 
        c = c.squeeze(1)
        c = self.composition_net(c)
        return c

class composition_CNN(nn.Module):
    def __init__(self, in_channels, output_dim):
        super(composition_CNN, self).__init__()
        self.flatten = nn.Flatten()

        self.composition_convnet = nn.Sequential(
            nn.Conv2d(1, 20, 5, 1, 0),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Conv2d(20, 10, 3, 1, 0),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Conv2d(10, 5, 3, 1, 0),
            nn.ReLU(),
            nn.BatchNorm2d(5),
        )

        self.flattened_size2 = self._get_flattened_comp_size(input_shape=(1, 1, 21, 100))

        self.post_composition_convnet = nn.Sequential(
                nn.Linear(self.flattened_size2, 230),
                nn.ReLU(),
                nn.Linear(230, 230),
                nn.ReLU(),
                nn.Linear(230, 230),
        )

    def _get_flattened_comp_size(self, input_shape):
        dummy_input = torch.zeros(input_shape)

        with torch.no_grad():
            dummy_output = self.composition_convnet(dummy_input)
        
        return int(np.prod(dummy_output.shape))
    
    def forward(self, c): 
        c = self.composition_convnet(c)
        c = self.flatten(c)
        c = self.post_composition_convnet(c)
        return c


class SimpleConvNet(nn.Module):
    def __init__(self, in_channels, output_dim, xrd_type = "cnn", composition_type = "mlp"):
        super(SimpleConvNet, self).__init__()

        if xrd_type == "cnn":
            self.xrd_module = XRD_convnet(in_channels, output_dim)
        else:
            self.xrd_module = None
        
        if composition_type == "mlp":
            self.composition_module = composition_MLP()
        elif composition_type == "cnn":
            self.composition_module = composition_CNN()
        else:
            self.composition_module = None
    
        if (xrd_type is not None) and (composition_type is not None):
            self.merge_net = nn.Sequential(
                nn.Linear(460, 230),
                nn.ReLU(),
                nn.Linear(230, 230)
            )

    def forward(self, x, c, nettype = None):
        if self.xrd_module: 
            x = self.xrd_module(x)
    
        if self.composition_module:
            c = self.composition_module(c)

        if self.merge_net:
            m = torch.cat((x, c), dim = 1)
            m = self.merge_net(m)
            return m
        
        elif self.xrd_module:
            return x
        elif self.composition_module:
            return c    
    
# Create the model instance and move it to the selected device
output_dim = 230  # Output dimension
model = SimpleConvNet(in_channels=1, output_dim=output_dim).to(device)

# Define optimizer and loss function
weight_decay = 0  # Example value, adjust based on your needs
optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [81]:
train_accuracy_log = []
train_loss_log = []
val_accuracy_log = []
val_loss_log = []
mode = "xrd"
for epoch in range(100):
    model.train()  # Set the model to training mode
    total_train_loss = 0
    correct_train = 0
    total_train = 0

    for xrd, composition, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(xrd, composition, mode)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += targets.size(0)
        correct_train += (predicted == targets).sum().item()

    train_accuracy = 100 * correct_train / total_train
    
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_valid_loss = 0
    correct_valid = 0
    total_valid = 0

    with torch.no_grad():  # No gradients needed for validation
        for xrd, composition, targets in valid_loader:
            outputs = model(xrd, composition, mode)
            loss = criterion(outputs, targets)
            total_valid_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 
            total_valid += targets.size(0)
            correct_valid += (predicted == targets).sum().item()

    valid_accuracy = 100 * correct_valid / total_valid

    total_train_loss = total_train_loss / len(train_loader)
    validation_loss = total_valid_loss / len(valid_loader)

    print(f"Epoch {epoch+1}, Training Loss: {total_train_loss}, Training Accuracy: {train_accuracy}%, Validation Loss: {validation_loss}, Validation Accuracy: {valid_accuracy}%")

    train_accuracy_log.append(train_accuracy)
    train_loss_log.append(total_train_loss)

    val_accuracy_log.append(valid_accuracy)
    val_loss_log.append(validation_loss)

Epoch 1, Training Loss: 3.352157087190777, Training Accuracy: 25.967894239848913%, Validation Loss: 2.8495099544525146, Validation Accuracy: 32.43422507185496%
Epoch 2, Training Loss: 2.3561569122557944, Training Accuracy: 42.279064600344384%, Validation Loss: 2.0758049223158093, Validation Accuracy: 47.99911563121822%


KeyboardInterrupt: 

In [ ]:
train_fraction_val_accuracy_informal = {0.2: (95.22, 47.33),
                                        0.4: (94.785, 57.373),
                                        0.7: (94.165, 65.001),
                                        1: (94.07876, 68.295)}